In [4]:
!pip install langchain langchain-community transformers sentence-transformers faiss-cpu streamlit pyngrok torch

In [1]:
# Install required libraries, including langchain-community
!pip install langchain langchain-community transformers sentence-transformers faiss-cpu streamlit pyngrok torchfrom langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os

# Create a sample corpus (simulating Wikipedia-style documents)
sample_corpus = """
Document 1: Artificial Intelligence Overview
Artificial Intelligence (AI) is the simulation of human intelligence by machines. It includes tasks like learning, reasoning, and problem-solving. Modern AI systems use machine learning, neural networks, and deep learning. Popular applications include image recognition, natural language processing, and autonomous vehicles.

Document 2: Machine Learning
Machine Learning (ML) is a subset of AI that focuses on algorithms learning from data. Supervised learning uses labeled data, while unsupervised learning finds patterns in unlabeled data. Common algorithms include linear regression, decision trees, and support vector machines.

Document 3: Deep Learning
Deep Learning is a type of ML using neural networks with many layers. It excels in tasks like image and speech recognition. Frameworks like TensorFlow and PyTorch are widely used. Deep learning requires large datasets and significant computational power.
"""

# Save corpus to a file
os.makedirs('/content/corpus', exist_ok=True)
with open('/content/corpus/ai_knowledge.txt', 'w') as f:
    f.write(sample_corpus)

print("Sample corpus created at /content/corpus/ai_knowledge.txt")

Sample corpus created at /content/corpus/ai_knowledge.txt


In [2]:
# Load the corpus
loader = TextLoader('/content/corpus/ai_knowledge.txt')
documents = loader.load()

# Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vector_store = FAISS.from_documents(docs, embeddings)

print(f"Created vector store with {len(docs)} document chunks")

/tmp/ipython-input-589684972.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Created vector store with 3 document chunks


In [6]:
import torch  # Added to fix NameError
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Load a lightweight LLM (distilgpt2 for Colab compatibility)
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a text generation pipeline
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    truncation=True,
    device=0 if torch.cuda.is_available() else -1
)

# Wrap in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# Set up conversational memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create conversational retrieval chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
    memory=memory
)

print("Conversational retrieval chain initialized")

Device set to use cuda:0


Conversational retrieval chain initialized


/tmp/ipython-input-249597988.py:23: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_gen_pipeline)
/tmp/ipython-input-249597988.py:26: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [7]:
# Test the chatbot with a few queries
queries = [
    "What is Artificial Intelligence?",
    "How does Machine Learning relate to AI?",
    "Tell me more about Deep Learning."
]

# Run queries and maintain context
for query in queries:
    response = qa_chain({"question": query})
    print(f"\nQuery: {query}")
    print(f"Response: {response['answer'].strip()}")

# Check memory (recent chat history)
print("\nChat history:")
print(memory.load_memory_variables({})['chat_history'])

/tmp/ipython-input-4113791076.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": query})
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Query: What is Artificial Intelligence?
Response: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Document 1: Artificial Intelligence Overview
Artificial Intelligence (AI) is the simulation of human intelligence by machines. It includes tasks like learning, reasoning, and problem-solving. Modern AI systems use machine learning, neural networks, and deep learning. Popular applications include image recognition, natural language processing, and autonomous vehicles.

Document 2: Machine Learning
Machine Learning (ML) is a subset of AI that focuses on algorithms learning from data. Supervised learning uses labeled data, while unsupervised learning finds patterns in unlabeled data. Common algorithms include linear regression, decision trees, and support vector machines.

Question: What is Artificial Intelligence?
Helpful Answer: Yes, Artificial Intelligence can become a valua

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Query: How does Machine Learning relate to AI?
Response: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Document 1: Artificial Intelligence Overview
Artificial Intelligence (AI) is the simulation of human intelligence by machines. It includes tasks like learning, reasoning, and problem-solving. Modern AI systems use machine learning, neural networks, and deep learning. Popular applications include image recognition, natural language processing, and autonomous vehicles.

Document 2: Machine Learning
Machine Learning (ML) is a subset of AI that focuses on algorithms learning from data. Supervised learning uses labeled data, while unsupervised learning finds patterns in unlabeled data. Common algorithms include linear regression, decision trees, and support vector machines.

Question: Given the following conversation and a follow up question, rephrase the follow up questio

In [8]:
%%writefile app.py
import streamlit as st
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Initialize session state for memory
if 'memory' not in st.session_state:
    # Load corpus
    loader = TextLoader('/content/corpus/ai_knowledge.txt')
    documents = loader.load()

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)

    # Create embeddings and vector store
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(docs, embeddings)

    # Initialize LLM
    model_name = "distilgpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    text_gen_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=100,
        truncation=True,
        device=0 if torch.cuda.is_available() else -1
    )
    llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

    # Set up memory
    st.session_state.memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )

    # Create conversational chain
    st.session_state.qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
        memory=st.session_state.memory
    )

# Streamlit app
st.title("Context-Aware AI Chatbot")
st.write("Ask questions about AI, Machine Learning, or Deep Learning. The chatbot uses a knowledge base and remembers our conversation.")

# Input query
user_input = st.text_input("Your question:", "")

if st.button("Ask"):
    if user_input:
        # Get response
        response = st.session_state.qa_chain({"question": user_input})
        st.write(f"**Response**: {response['answer'].strip()}")

        # Display chat history
        st.subheader("Chat History")
        history = st.session_state.memory.load_memory_variables({})['chat_history']
        for i, msg in enumerate(history):
            if i % 2 == 0:
                st.write(f"**You**: {msg.content}")
            else:
                st.write(f"**Bot**: {msg.content}")
    else:
        st.write("Please enter a question.")

Writing app.py


In [10]:
from pyngrok import ngrok

# Set ngrok authtoken (replace with your actual token from ngrok.com)
!ngrok authtoken 31onOyFv68hlV5TS5ULVJEDNJFU_5mzRkrTDsJTcWbY32XCb

# Stop any running Streamlit process
!pkill streamlit

# Start Streamlit server
!streamlit run app.py &>/dev/null &

# Create public URL
ngrok.kill()  # Close existing tunnels
public_url = ngrok.connect(8501)
print(f"Streamlit app running at: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app running at: NgrokTunnel: "https://fca2e8edd865.ngrok-free.app" -> "http://localhost:8501"


In [11]:
# Stop Streamlit and ngrok
!pkill streamlit
ngrok.kill()
print("Streamlit and ngrok stopped")

Streamlit and ngrok stopped
